## Convergence analysis
This notebook is to determine if we can replicate the convergence results from the King et al. 2023 paper, using the new datasets, and the new models. 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import cortico_cereb_connectivity.globals as gl
import cortico_cereb_connectivity.run_model as rm
import cortico_cereb_connectivity.cio as cio
import matplotlib.pyplot as plt
import seaborn as sb
import scipy.stats as stats

fig_dir =  '/Users/jdiedrichsen/Dropbox/Talks/2024/12_UTSW'